In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install scikit-plot

In [ ]:
import pandas as pd
import io
import matplotlib.pyplot as plt
import numpy as np
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.preprocessing import Normalizer
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.preprocessing import MinMaxScaler 
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import Flatten
from keras.constraints import maxnorm
from keras.utils.vis_utils import plot_model
import tensorflow as tf
np.random.seed(1337)  # for reproducibility
import sklearn.metrics as metrics
from sklearn.metrics.classification import accuracy_score
from keras.layers import Dense, Dropout, Activation, Embedding, Flatten, LSTM
#from dbn.tensorflow import SupervisedDBNClassification
import h5py
from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay
from sklearn.metrics import (precision_score, recall_score,f1_score, accuracy_score,mean_squared_error,mean_absolute_error,roc_auc_score,cohen_kappa_score)
from sklearn.model_selection import KFold,cross_val_score
import matplotlib.pyplot as plt
import scikitplot as skplt
kfold=KFold(5,True,3)
cvscores = []

/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.metrics.classification module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.metrics. Anything that cannot be imported from sklearn.metrics is now part of the private API.
  warnings.warn(message, FutureWarning)


In [ ]:
#### DNN #######
def cleaningdata(df):
    assert isinstance(df, pd.DataFrame), "df needs to be a pd.DataFrame"
    df.dropna(inplace=True)
    indices_to_keep = ~df.isin([np.nan, np.inf, -np.inf]).any(1)
    return df[indices_to_keep].astype(np.float64)
# fetching dataset from path
dataset = pd.read_csv('/content/drive/MyDrive/IoT Intrusion Dataset 2020.csv', encoding='utf-8')
# for object type data we will label it and transform into apporopriate type fo data after using fit_transform on that colomn to avoid errors
for column in dataset.columns:
    if dataset[column].dtype == type(object):
        le = LabelEncoder()
        dataset[column] = le.fit_transform(dataset[column])
data = cleaningdata(dataset)

# collecting all required colomns in data to X
X = data.iloc[:, 0:85].values

# collecting our LABEL colomn in y
y = data.iloc[:, -2].values

from sklearn.preprocessing import label_binarize
Y = label_binarize(y, classes=[0, 1, 2, 3, 4])
n_classes = Y.shape[1]
print(n_classes)

for train, test in kfold.split(X,y):
   X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=52)
   from keras.utils.np_utils import to_categorical
   sc = StandardScaler()
   X_train = sc.fit_transform(X_train)
   X_test = sc.transform(X_test)

   #X_train.shape

   X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1]))
   X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1]))

   x_train = np.array(X_train)
   x_test = np.array(X_test)
   y_train1 = np.array(y_train)
   y_test1 = np.array(y_test)

   y_train= to_categorical(y_train1)
   y_test= to_categorical(y_test1)
# 1. define the network
   model = Sequential()
   model.add(Dense(800, input_dim=85, activation='relu'))
   model.add(Dropout(0.8))
   model.add(Dense(400, activation='relu'))
   model.add(Dropout(0.8))
   model.add(Dense(5,activation='softmax'))
   model.compile(optimizer='adam', loss = 'categorical_crossentropy', metrics=['accuracy','mae'])
   history = model.fit(X_train, y_train, epochs = 10, batch_size=64, verbose=1,shuffle=False)
   scores = model.evaluate(X_test, y_test, verbose=1)
   print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))
   cvscores.append(scores[1] * 100)
print("%.2f%% (+/- %.2f%%)" % (np.mean(cvscores), np.std(cvscores)))


5
Epoch 1/10
7818/7818 [==============================] - 52s 7ms/step - loss: 0.0535 - accuracy: 0.9913 - mae: 0.0054
Epoch 2/10
7818/7818 [==============================] - 51s 7ms/step - loss: 0.0307 - accuracy: 0.9979 - mae: 0.0011
Epoch 3/10
7818/7818 [==============================] - 51s 7ms/step - loss: 0.0294 - accuracy: 0.9981 - mae: 9.0100e-04
Epoch 4/10
7818/7818 [==============================] - 52s 7ms/step - loss: 0.0253 - accuracy: 0.9983 - mae: 7.9844e-04
Epoch 5/10
7818/7818 [==============================] - 52s 7ms/step - loss: 0.0190 - accuracy: 0.9986 - mae: 6.7292e-04
Epoch 6/10
7818/7818 [==============================] - 52s 7ms/step - loss: 0.0229 - accuracy: 0.9986 - mae: 6.6182e-04
Epoch 7/10
7818/7818 [==============================] - 53s 7ms/step - loss: 0.0330 - accuracy: 0.9985 - mae: 6.6766e-04
Epoch 8/10
7818/7818 [==============================] - 52s 7ms/step - loss: 0.0231 - accuracy: 0.9987 - mae: 5.8021e-04
Epoch 9/10
7818/7818 [================

In [ ]:
target_names = ['Mirai', 'DoS', 'Scan', 'Normal', 'MITM_ARP_Spoofing']

In [ ]:
### train_accuracy
from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay
from sklearn.metrics import (precision_score, recall_score,f1_score, accuracy_score,mean_squared_error,mean_absolute_error,roc_auc_score,cohen_kappa_score)
import sklearn.metrics as metrics

expected = y_train
predicted = model.predict(X_train)
data = classification_report(expected,np.round(predicted), target_names=target_names)
print(data)


print("Accuracy_Train:",metrics.accuracy_score(expected, np.round(predicted)))
print("Precision_Train:",metrics.precision_score(expected, np.round(predicted), average='weighted'))
print("Recall_Train:",metrics.recall_score(expected, np.round(predicted), average='weighted'))
print("f1-score_Train:",metrics.f1_score(expected, np.round(predicted), average='weighted'))
print("ROC_AUC_Train:",metrics.roc_auc_score(expected, predicted,multi_class="ovr",average='weighted'))

                   precision    recall  f1-score   support

            Mirai       1.00      1.00      1.00     47576
              DoS       1.00      1.00      1.00     28268
             Scan       1.00      1.00      1.00    332252
           Normal       1.00      1.00      1.00     32013
MITM_ARP_Spoofing       1.00      1.00      1.00     60223

        micro avg       1.00      1.00      1.00    500332
        macro avg       1.00      1.00      1.00    500332
     weighted avg       1.00      1.00      1.00    500332
      samples avg       1.00      1.00      1.00    500332

Accuracy_Train: 0.9995902720593526
Precision_Train: 0.9995905328397802
Recall_Train: 0.9995902720593526
f1-score_Train: 0.9995902550170359
ROC_AUC_Train: 0.9998935576179743


In [ ]:
expected = y_test
predicted = model.predict(X_test)
data = classification_report(expected,np.round(predicted), target_names=target_names)
print(data)


print("Accuracy:",metrics.accuracy_score(expected, np.round(predicted)))
print("Precision:",metrics.precision_score(expected, np.round(predicted), average='weighted'))
print("Recall:",metrics.recall_score(expected, np.round(predicted), average='weighted'))
print("f1-score:",metrics.f1_score(expected, np.round(predicted), average='weighted'))
print("ROC_AUC:",metrics.roc_auc_score(expected, predicted,multi_class="ovr",average='weighted'))

                   precision    recall  f1-score   support

            Mirai       1.00      1.00      1.00     11815
              DoS       1.00      1.00      1.00      7109
             Scan       1.00      1.00      1.00     83057
           Normal       1.00      1.00      1.00      8060
MITM_ARP_Spoofing       1.00      1.00      1.00     15042

        micro avg       1.00      1.00      1.00    125083
        macro avg       1.00      1.00      1.00    125083
     weighted avg       1.00      1.00      1.00    125083
      samples avg       1.00      1.00      1.00    125083

Accuracy: 0.99953630789156
Precision: 0.9995368440848069
Recall: 0.99953630789156
f1-score: 0.9995363126395711
ROC_AUC: 0.9999021783115638
